In [1]:
import json
from pprint import pprint
from fuzzywuzzy import fuzz
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter

/home/kirigaya/.local/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
# clear data   O(n)
def remove_abnormal(text):
    result = ""
    enter = False
    for i, c in enumerate(text):
        if enter:
            if c != '"':
                result += c
            else:
                if text[i + 1] in ["}"] or text[i + 1: i + 3] in ["],", "]}"] or text[i + 1: i + 4] in [', "', ': "', ': [']:
                    result += c
                    enter = False
        else:
            result += c
            if i > 2:
                pre_str = text[i - 2: i + 1]
                if pre_str in [': "', ', "']:
                    enter = True
        
    return result

wfp = open("data/results.json", "w", encoding="utf-8")
for line in open("data/data.json", "r", encoding="utf-8"):
    line = remove_abnormal(line.strip())
    wfp.write(line)
    wfp.write("\n")
wfp.close()

In [16]:
# add relation    O(n^2)
all_data = [json.loads(line.strip()) for line in open("data/data_step1.json", "r", encoding="utf-8")]

all_data[0]

{'title': 'Winter Olympic Games',
 'url': 'https://en.wikipedia.org/wiki/Winter_Olympic_Games',
 'desc': "The Winter Olympic Games (French: Jeux olympiques d'hiver)[note 1] is a major international multi-sport event held once every four years for sports practiced on snow and ice. The first Winter Olympic Games, the 1924 Winter Olympics, were held in Chamonix, France. The modern Olympic Games were inspired by the ancient Olympic Games, which were held in Olympia, Greece, from the 8th century BC to the 4th century AD. Baron Pierre de Coubertin founded the International Olympic Committee (IOC) in 1894, leading to the first modern Summer Olympic Games in Athens, Greece in 1896. The IOC is the governing body of the Olympic Movement, with the Olympic Charter defining its structure and authority.",
 'time': [],
 'place': [],
 'joiner': [{'type': 'GPE', 'content': 'Norway'},
  {'type': 'GPE', 'content': 'Russia'},
  {'type': 'GPE', 'content': 'the United States'},
  {'type': 'GPE', 'content': 

In [46]:
def get_sim_set(data_index : int, score_thred : int = 90) -> list:
    sim_set = []
    for i in range(len(all_data)):
        if i == data_index:
            continue
        score = fuzz.token_set_ratio(all_data[data_index]["title"], all_data[i]["desc"])
        if score >= score_thred:
            sim_set.append(all_data[i]["title"])
    return sim_set

sim_set = get_sim_set(2)
print(sim_set)

['Youth Olympic Games']


In [47]:
nums = []
for i in tqdm(range(len(all_data))):
    sim_set_num = len(get_sim_set(i))
    nums.append(sim_set_num)

100%|██████████| 4090/4090 [17:23<00:00,  3.92it/s]


NameError: name 'sns' is not defined

In [59]:
Counter(nums)

Counter({32: 1,
         24: 22,
         1: 521,
         18: 35,
         0: 2550,
         14: 11,
         63: 1,
         5: 80,
         52: 2,
         23: 5,
         2: 192,
         10: 30,
         8: 47,
         7: 51,
         6: 40,
         13: 30,
         4: 59,
         3: 132,
         161: 1,
         33: 1,
         9: 22,
         16: 35,
         11: 11,
         15: 21,
         12: 10,
         25: 28,
         19: 10,
         17: 18,
         20: 23,
         26: 8,
         21: 6,
         48: 1,
         27: 4,
         70: 56,
         44: 1,
         71: 2,
         43: 12,
         22: 2,
         45: 1,
         30: 1,
         29: 2,
         28: 1,
         57: 1,
         108: 1,
         72: 1,
         64: 1})

In [64]:
fp = open("data/data_step2.json", "w", encoding="utf-8")
for i in tqdm(range(len(all_data))):
    sim_set = get_sim_set(i)
    all_data[i]["link"] = sim_set
    fp.write(json.dumps(all_data[i], ensure_ascii=False))
    fp.write("\n")
fp.close()

100%|██████████| 4090/4090 [17:25<00:00,  3.91it/s]
